In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import datetime
import time
%matplotlib inline
pd.options.display.max_columns = None

In [2]:
start = time.time()
df=pd.read_csv("TOTAL.csv",encoding="UTF-8")
#物件重複回避
df_bukken=df[~df.duplicated(subset="cin_bukken_cd")]
#pv,cvsumをつくる
bukken_pv=df.groupby("cin_bukken_cd")["pv_num"].sum()
df_bukken_pvnum=pd.DataFrame(bukken_pv).reset_index()
df_bukken_pvnum.columns=["cin_bukken_cd","pv_sum"]
bukken_cv=df.groupby("cin_bukken_cd")["cv_num"].sum()
df_bukken_cvnum=pd.DataFrame(bukken_cv).reset_index()
df_bukken_cvnum.columns=["cin_bukken_cd","cv_sum"]
#sumを物件にmerge
df_bukken=df_bukken.drop(["Unnamed: 0","Unnamed: 0.1","ymd"],axis=1)
df_bukken_cvsum=pd.merge(df_bukken,df_bukken_cvnum,on="cin_bukken_cd",how="left")
df_bukken_pvcv=pd.merge(df_bukken_cvsum,df_bukken_pvnum,on="cin_bukken_cd",how="left")
#期間割り出し
df_bukken_pvcv["start"]=pd.to_datetime(df_bukken_pvcv["keisai_start"],format="%Y-%m-%d")
df_bukken_pvcv["end"]=pd.to_datetime(df_bukken_pvcv["keisai_end"],format="%Y-%m-%d")
df_bukken_pvcv["keisai_kikan"]=0
df_bukken_pvcv["keisai_kikan"]=df_bukken_pvcv["keisai_kikan"]+df_bukken_pvcv.end-df_bukken_pvcv.start+datetime.timedelta(days=1)
df_bukken_pvcv["keisai_kikan"]=df_bukken_pvcv["keisai_kikan"].astype("timedelta64[D]")
df_bukken_pvcv_kikan=df_bukken_pvcv
#real_lgbtリスト作成してmerge
df_choice=df_bukken_pvcv_kikan.loc[:,['perm_juko_nayose_cd', 'lgbt_flg']]
df_choice_sort=df_choice.sort_values(['perm_juko_nayose_cd', 'lgbt_flg'], ascending=[True, True])
df_choice_sort.columns=["perm_juko_nayose_cd","real_lgbt_flg"]
df_choice_lgbtflg=df_choice_sort.drop_duplicates(subset='perm_juko_nayose_cd', keep='last')
df_bukken_pvcv_kikan_lgbtflg=pd.merge(df_bukken_pvcv_kikan,df_choice_lgbtflg,on="perm_juko_nayose_cd",how="left")
#一旦savepoint
df_bukken_pvcv_kikan_lgbtflg.to_csv("TOTAL_pre_nayose.csv")
#再度読み込み
nayoseyou=pd.read_csv("TOTAL_pre_nayose.csv",encoding="UTF-8")
#pv,cv,kikanの住戸のsum
nayoseyou_pv=nayoseyou.groupby("perm_juko_nayose_cd")["pv_sum"].sum()
nayoseyou_pvsum=pd.DataFrame(nayoseyou_pv).reset_index()
nayoseyou_pvsum.columns=["perm_juko_nayose_cd","pv_zyuukosum"]
nayoseyou_cv=nayoseyou.groupby("perm_juko_nayose_cd")["cv_sum"].sum()
nayoseyou_cvsum=pd.DataFrame(nayoseyou_cv).reset_index()
nayoseyou_cvsum.columns=["perm_juko_nayose_cd","cv_zyuukosum"]
nayoseyou_keisai=nayoseyou.groupby("perm_juko_nayose_cd")["keisai_kikan"].sum()
nayoseyou_keisaisum=pd.DataFrame(nayoseyou_keisai).reset_index()
nayoseyou_keisaisum.columns=["perm_juko_nayose_cd","keisai_zyuukosum"]
#nayoseから重複を消す
nayoseyou=nayoseyou[~nayoseyou.duplicated(subset="perm_juko_nayose_cd")]
#nayoseを加工
nayoseyou1=nayoseyou.drop(['Unnamed: 0', 'cin_bukken_cd', 'lgbt_flg','pv_num', 'cv_num', 'keisai_start', 'keisai_end',"start","end","cv_sum","pv_sum","keisai_kikan"],axis=1)
#pv,cv,kikansumをmerge
new_nayose=pd.merge(nayoseyou1,nayoseyou_pvsum,on="perm_juko_nayose_cd",how="left")
new_nayose=pd.merge(new_nayose,nayoseyou_cvsum,on="perm_juko_nayose_cd",how="left")
new_nayose=pd.merge(new_nayose,nayoseyou_keisaisum,on="perm_juko_nayose_cd",how="left")
#平均を出す
new_nayose["ave_pv"]=new_nayose["pv_zyuukosum"]/new_nayose["keisai_zyuukosum"]
new_nayose["ave_cv"]=new_nayose["cv_zyuukosum"]/new_nayose["keisai_zyuukosum"]
#名前を変更
new_nayose.columns=['perm_juko_nayose_cd', 'KISS_CD', 'KAISU', 'CIN_CHIJOKAIDATE',
       'CIN_CHIKAKAIDATE', 'KAISO_DISP', 'GOOGLE_STREETVIEW_FLG',
       'CHIKU_NEN_TSUKI_DISP', 'CHIKU_NEN_TSUKI', 'SHIKUGUN_CD', 'shikugun_nm',
       'OOAZA_CD', 'ooaza_nm', 'ENSEN_CD_1', 'CIN_ENSEN_NM_1', 'CIN_EKI_NM_1',
       'KOTSU_KBN_1', 'TOHO_JIKAN_1', 'TOHO_JIKAN_DISP_1', 'BUS_JIKAN_DISP_1',
       'BUS_JIKAN_1', 'KOTSU_KANREN_1', 'KOTSU_DISP_1', 'ENSEN_CD_2',
       'CIN_ENSEN_NM_2', 'CIN_EKI_NM_2', 'KOTSU_KBN_2', 'TOHO_JIKAN_2',
       'TOHO_JIKAN_DISP_2', 'BUS_JIKAN_DISP_2', 'BUS_JIKAN_2',
       'KOTSU_KANREN_2', 'KOTSU_DISP_2', 'ENSEN_CD_3', 'CIN_ENSEN_NM_3',
       'CIN_EKI_NM_3', 'KOTSU_KBN_3', 'TOHO_JIKAN_3', 'TOHO_JIKAN_DISP_3',
       'BUS_JIKAN_DISP_3', 'BUS_JIKAN_3', 'KOTSU_KANREN_3', 'KOTSU_DISP_3',
       'CHINRYO', 'CIN_CHINRYO_DISP', 'CHUSHAJO_RYO', 'CIN_KANRIHI',
       'REIKIN_DISP', 'SHIKIKIN_DISP', 'SHOKYAKUKIN_DISP', 'SHIKIBIKI_DISP',
       'HOSHOKIN_DISP', 'HOKA_SHOKI_HIYO_DISP', 'CIN_SONOTA_SHOHIYO',
       'HEYA_SU', 'TATEMONO_SHU_CD', 'KOZO_SHU_CD', 'KOZO_DISP', 'MADORI_TYPE',
       'MADORI_SHOSAI_KBN', 'MENSEKI', 'MENSEKI_DISP', 'CIN_TSUBOTANKA',
       'BAL_MENSEKI_DISP', 'KAIKO_MUKI', 'KAIKO_MUKI_DISP', 'NYUKYO_KBN',
       'TESURYO_RITSU', 'TESURYO_RITSU_DISP', 'SETSUBI_DISP',
       'STB_SHUHEN_UMU_FLG', 'REIKIN_SHIKIBIKI_DISP', 'SHIKIKIN_HOSHOKIN_DISP',
       'TEIKI_SHAKKA', 'real_lgbt_flg', 'all_pv', 'all_cv',
       'all_kikan', 'ave_pv', 'ave_cv']
#名寄せのデータを完成
new_nayose.to_csv("nayose_TOTAL.csv")

FileNotFoundError: File b'TOTAL.csv' does not exist